# Graphito: ¿Podemos decir que FIUBA es una red social?

A lo largo del trabajo práctico vamos a generar diferentes grafos con el set de datos de alumnos de FIUBA. Pero para dar un primer pantallazo, vamos a conectar a aquellos alumnos que cursaron juntos alguna materia en un mismo cuatrimestre y vamos a hacer un análisis general de la misma:
- ¿Cuál es su diámetro(largo máximo de todos los caminos mínimos)?
- ¿Cuál es el grado promedio? ¿Les alumnes tienden a tener muchas conexiones, es decir que tienden a hacer las mismas materias con las mismas personas? ¿Cómo se distribuyen los grados?
- ¿Cuál es el coeficiente de clustering? ¿Mis amigues son amigues entre sí?
- ¿Tiene una componente gigante?
- ¿Es una red aleatoria? 

In [ ]:
import pandas as pd
import utils

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
df_nodos = df[df.columns & ["Padron", "materia_id", "materia_cuatrimestre"]]

# Sacamos a aquellos que no ordenan su carrera por cuatrimestres
df_nodos.dropna(subset=['materia_cuatrimestre'], inplace=True)
# Sacamos a aquellos que solo agregaron 1 o 2 cuatrimetres a modo de prueba
df_nodos = df_nodos.groupby(['Padron', 'materia_cuatrimestre']).filter(lambda x: len(x)>2)

df_nodos.sample(3)

In [ ]:
df_red = utils.construir_df_pareando_padrones_por(df_nodos, 'materia_cuatrimestre')

df_red = df_red.drop_duplicates(['Padron_min', 'Padron_max']).reset_index()
df_red = df_red[['Padron_x', 'Padron_y', 'materia_id', 'materia_cuatrimestre']]
df_red.rename(columns = {'Padron_x':'src_padron', 'Padron_y':'dst_padron'}, inplace = True)

display(df_red.sample(3))
df_red.shape

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df_red, 
                            source='src_padron', 
                            target='dst_padron',
                            create_using=nx.Graph())

nx.set_node_attributes(G, df_red.to_dict('index'))

print(G)

In [ ]:
print(f"""El \033[1mdiámetro\033[0m de nuestra red es de {nx.diameter(G)}, y tiene mucho sentido que sea un número tan chico porque el Fiuba Map tiene cargadas las notas de alumnos desde el 2014 en adelante (aproximadamente), así que no hay tantas camadas de alumnos, sumado a que estamos filtrando por una carrera. Además de esto, los alumnos avanzan en la carrera a diferentes velocidades, por lo que hay recursantes que cursan con varias camadas. Teniendo en cuenta estos tres factores, hace que la mayoría de los nodos en el grafo están muy conectados. Es interesante también ver que la \033[1mdistancia promedio de la red\033[0m de nuestra red es de {nx.average_shortest_path_length(G)}, lo cual se relaciona mucho a tener un diámetro tan pequeño.\n""")

utils.plot_diametro(G)

In [ ]:
print(f"""El \033[1mgrado promedio de la red\033[0m es de {sum([n[1] for n in G.degree()]) / len(G):.2f}. Sin embargo la distribución de los grados es uniforme, y eso se debe a que por un lado tenemos alumnes con gran antigüedad(los que escriben este TP) y por otro lado alumnes que recién arrancan las cursadas. Es interesante pensar, que a medida que los años pasen, la distribución va a inclinarse más a los números altos (que es algo que ya podemos notar).\n""")

utils.plot_distribucion_grados([G])

In [ ]:
print(f"""El \033[1mclustering\033[0m es la probabilidad de que dos vértices adyacentes de A sean adyacentes entre sí. El clustering promedio de esta red es {"%3.4f"%nx.average_clustering(G)}. En redes sociales tenemos en general un coeficiente de Clustering alto, aca podemos notar uno bastante alto. Un ejemplo claro es nuestra bella amistad con uno de los integrantes del trabajo. Un amigue A curso con B y nos presentó a B, desde entonces cursamos todas las materias juntes. """)

utils.plot_clustering(G)

In [ ]:
print(f"""Por último, la componente {"es conexa" if nx.is_connected(G) else "no es conexa"}, es decir que tenemos una \033[1mcomponente gigante\033[0m.\n""")

Ahora, vamos a realizar una simulación de un modelado de **Erdös-Rényi** y un modelado de **Preferential Attachment** (ley de potencias) que correspondan a los parámetros de nuestra red, para luego poder compararlos y poder entender si nuestra red es aleatoria. Para eso vamos a conocer el diámetro, grado promedio, distancia promedio, clustering promedio y si se trata de una componente gigante. Al finalizar, vamos realizar una representación de anonymous walks para cada una de las redes y vamos a determinar por distancia coseno cuál sería la simulación más afín y por qué.

> El modelo Erdös-Rényi se utiliza para crear redes aleatorias en las redes sociales. En el modelo Erdös-Rényi, se tiene que un nuevo nodo se enlaza con igual probabilidad con el resto de la red, es decir posee una independencia estadística con el resto de nodos de la red

> Para el modelo de Preferential Attachment usamos un algoritmo que se denomina Modelo de Barabási–Albert el cual genera redes aleatorias complejas empleando una regla o mecanismo denominado conexión preferencial. Las redes generadas por este algoritmo poseen una distribución de grado de tipo potencia.

In [ ]:
g_erdos = nx.erdos_renyi_graph(G.number_of_nodes(), 0.6)
print(f"""
Red aleatoria Erdös-Renyi
  El diámetro de la red: {nx.diameter(g_erdos)}
  El grado promedio de la red: {sum([n[1] for n in g_erdos.degree()]) / len(g_erdos):.2f}
  La distancia promedio de la red: {nx.average_shortest_path_length(g_erdos)}
  Clustering promedio: {"%3.4f"%nx.average_clustering(g_erdos)}
  Puentes globales: {list(nx.bridges(g_erdos))}
  {"Es conexa" if nx.is_connected(g_erdos) else "No es conexa"}
""")

In [ ]:
g_preferential_attachment = nx.barabasi_albert_graph(G.number_of_nodes(), G.number_of_nodes()//G.number_of_nodes())
print(f"""
Red aleatoria Preferential Attachment
  El diámetro de la red: {nx.diameter(g_preferential_attachment)}
  El grado promedio de la red: {sum([n[1] for n in g_preferential_attachment.degree()]) / len(g_preferential_attachment):.2f}
  La distancia promedio de la red: {nx.average_shortest_path_length(g_preferential_attachment)}
  Clustering promedio: {nx.average_clustering(g_preferential_attachment)}
  {"Es conexa" if nx.is_connected(g_preferential_attachment) else "No es conexa"}
""")

In [ ]:
aw_erdos = utils.anoymous_walks(g_erdos)
aw_preferential = utils.anoymous_walks(g_preferential_attachment)
aw_original = utils.anoymous_walks(G)

utils.plot_anoymous_walks([aw_original, aw_erdos, aw_preferential])

In [ ]:
from numpy import linalg as LA
import numpy as np

print(f"""
Las leyes de potencias aparecen de la \033[1m ventaja acumulativa\033[0m. Esto puede verse como un desbalance desproporcionado entre los que tienen muchos contactos, y los que tienen pocos. Es claro que el grafon no iba a ser similar a el grafo generado por Barabási–Albert, ya que no tiene sentido que un alumno tenga muchísimas aristas, mientras que otros tengan pocas ya que hay muchas materias en común. La distancia de coseno entre nuestro grafon y el grafo aleatorio es de {1 - np.inner(aw_original[7],  aw_preferential[7]) / (LA.norm(aw_original[7]) * LA.norm( aw_preferential[7]))}. \n
En cambio, al compararlo con el grafo generado con Erdös Renyi, podemos encontrar más similitudes. A diferencia del anterior, todos los nodos tienen probabilidad de contar con muchas aristas, dejando un grafo más real y parecido a una red de una Universidad en donde todos los alumnos tienden a cursar materias similares. La distancia de coseno entre los mismos {1 - np.inner(aw_original[7],  aw_erdos[7]) / (LA.norm(aw_original[7]) * LA.norm(aw_erdos[7]))}
""")

In [ ]:
utils.plot_distribucion_grados([G, g_erdos, g_preferential_attachment])